# Pyplot

In [ ]:
%%spark
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
import numpy as np
import pandas as pd

def label(bin):
    return f"'({bin[0]:5.2f}, {bin[1]:5.2f}]'"

def hist_for_feature(df, name_col=None, bins=True, b=10, title=None, color='steelblue', data_labels=True, bar_offset=2.5, rotate_ticks=False, index_order=False, resample=True, ylab = 'Frequência'):
    
    if name_col is not None:
        k = list(df.columns).index(name_col)
    else:
        k=0
    
    # Create a figure
    fig, axs = plt.subplots(figsize=(20, 10))
    fig.set_facecolor('white')
        
    if bins:
        h = df.select(name_col).rdd.flatMap(lambda x: x).histogram(b) #change 3 to the number of expected intervals
        bins = [(x, h[0][i+1]) for i,x in enumerate(h[0][:-1])]
        
        e = "case "
        for bin in bins[:-1]:
            e += f"when {name_col} >= {bin[0]} and {name_col} < {bin[1]} then {label(bin)} "
        e += f"else {label(bins[-1])} end as bins"
        
        df = df.selectExpr(e)
    else:
        df = df.withColumn('bins', F.col(name_col))
    
    if resample:
        df = df.groupBy('bins').count().sort(F.col('bins').asc())
    else:
        df = df.drop(name_col)
        print('Usar dados já calculados...')
    
    df = df.toPandas()
    df = df.set_index('bins')
    
    if index_order:
        df = df.reindex(index=index_order).fillna(0)
    
    x_axis = [str(i) for i in df.index]
    
    # Create a secondary axis
    #axs_2 = axs.twinx()
    if title:
        axs.set_title(title, fontsize = 30)
    else:
        axs.set_title(f'Histograma de {name_col}', fontsize = 30)

    # Creating the bar plot
    plt.bar(x=x_axis, height=df[df.columns[0]], color = color, width = 0.4)

    # Clean axis labels
    axs.set_ylabel(ylab, fontsize = 15)
    #axs_2.set_ylabel('Count', fontsize= 15)

    if rotate_ticks:
        #axs.set_xticklabels(axs.get_xticks(), rotation = 45)
        axs.tick_params(axis='x', labelrotation = 45)
    
    axs.set_xlabel(f'{name_col}', fontsize = 15)
    
    if data_labels:
        rects = axs.patches
        col_values = df[df.columns[0]].reset_index(drop=True)
        labels = ['{:,}'.format(col_values[i]).replace(',','.') for i in range(len(rects))]

        for rect, label in zip(rects, labels):
            height = rect.get_height()
            axs.text(
                rect.get_x() + rect.get_width() / 2, height + bar_offset, label, ha="center", va="bottom", fontsize=10
            )

    
    return fig, df

UsageError: Cell magic `%%spark` not found.


In [ ]:
%%spark

def label(bin):
    return f"'({bin[0]:5.2f}, {bin[1]:5.2f}]'"

def pareto_for_feature(df, name_col=None, bins=True, b=10, title=None, color='steelblue', data_labels=True, bar_offset=2.5, rotate_ticks=False, color1 = 'steelblue', color2 = 'navy', sort_count=True):
    
    if name_col is not None:
        k = list(df.columns).index(name_col)
    else:
        k=0
    
    # Create a figure
    fig, axs = plt.subplots(figsize=(20, 10))
    fig.set_facecolor('white')
        
    if bins:
        h = grouped_sdf.select(name_col).rdd.flatMap(lambda x: x).histogram(b) #change 3 to the number of expected intervals
        bins = [(x, h[0][i+1]) for i,x in enumerate(h[0][:-1])]
        
        e = "case "
        for bin in bins[:-1]:
            e += f"when {name_col} >= {bin[0]} and {name_col} < {bin[1]} then {label(bin)} "
        e += f"else {label(bins[-1])} end as bins"
        
        df = df.selectExpr(e)
    else:
        df = df.withColumn('bins', F.col(name_col))
        
    df = df.groupBy('bins').count().sort(F.col('bins').asc())
    
    df = df.toPandas()
    df['bins'] = df['bins'].astype(str)
    df = df.set_index('bins')
    
    if sort_count:
        df = df.sort_values(by='count', ascending=False)
    
    x_axis = [str(i) for i in df.index]
    
    # sort DataFrame by count descending
    #df = df.sort_values(by=df.columns[0], ascending=False)

    #add column to display cumulative percentage
    df['cumperc'] = df[df.columns[0]].cumsum()/df[df.columns[0]].sum()

    #define aesthetics for plot
    line_size = 6

    # Create basic bar plot
    axs.bar(df.index, df[df.columns[0]], color=color1)

    #add cumulative percentage line to plot
    ax2 = axs.twinx()
    ax2.plot(df.index, df['cumperc'], color=color2, marker="D", ms=line_size)
    ax2.yaxis.set_major_formatter(PercentFormatter())

    #specify axis colors
    axs.tick_params(axis='y')#, colors=color1)
    ax2.tick_params(axis='y')#, colors=color2)

    for x_val, y_val in zip (range(len(df)), df["cumperc"]):
        # set the format of the text to %
        text = f"{y_val:.0%}"

        # place the text labels on the graph
        if y_val > .95:
            ax2.text(
                x=x_val+0.10,
                y=y_val-0.025,
                s=text, 
                fontsize=10,
                color="black",
                ha="center",
                va="center"
            )
        else:
            ax2.text(
                x=x_val-0.10,
                y=y_val+0.025,
                s=text, 
                fontsize=10,
                color="black",
                ha="center",
                va="center"
            )

    ax2.set_ylim(0, 1.01)
    ax2.set_yticks(np.arange(0, 1.05, 0.1))
    ax2.set_yticklabels([f"{y_val:.0%}" for y_val in np.arange(0, 1.05, 0.1)])
    
    # Create a secondary axis
    #axs_2 = axs.twinx()
    if title:
        axs.set_title(title, fontsize= 30)
    else:
        axs.set_title(f'Pareto de {name_col}', fontsize= 30)

    # Creating the bar plot
    #plt.bar(x=x_axis, height=df[df.columns[0]], color = color, width = 0.4)

    # Clean axis labels
    axs.set_ylabel('Frequência', fontsize= 15)
    ax2.set_ylabel('Percentual acumulado', fontsize= 15)
    
    axs.set_xlabel(f'{name_col}', fontsize= 15)
    
    if rotate_ticks:
        #axs.set_xticklabels(axs.get_xticks(), rotation = 45)
        axs.tick_params(axis='x', labelrotation = 45)
    
    if data_labels:
        rects = axs.patches
        col_values = df[df.columns[0]].reset_index(drop=True)
        labels = ['{:,}'.format(col_values[i]).replace(',','.') for i in range(len(rects))]

        for rect, label in zip(rects, labels):
            height = rect.get_height()
            axs.text(
                rect.get_x() + rect.get_width() / 2, height + bar_offset, label, ha="center", va="bottom", fontsize=10
            )

    
    return fig, df